## **Homework - Session 8**

## **Dataset**

In [2]:
# !wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip

## **Data preparation**

The dataset contains 12,500 images of cats and 12,500 images of dogs. 

Now we need to split this data into train and validation

* Create a `train` and `validation` folders
* In each folder, create `cats` and `dogs` folders
* Move the first 10,000 images to the train folder (from 0 to 9999) for boths cats and dogs - and put them in respective folders
* Move the remaining 2,500 images to the validation folder (from 10000 to 12499)

You can do this manually or with Python (check `os` and `shutil` packages).

In [5]:
# unzip train.zip
!unzip -q '/content/train.zip'

In [12]:
# create folders
!mkdir val
!mkdir train/dogs val/dogs train/cats val/cats

In [9]:
!ls

sample_data  train  train.zip


In [13]:
# move images
import os

for i in range(0, 10000):
    os.rename(f'train/dog.{i}.jpg', f'train/dogs/dog.{i}.jpg')
    os.rename(f'train/cat.{i}.jpg', f'train/cats/cat.{i}.jpg')

for i in range(10000, 12500):
    os.rename(f'train/dog.{i}.jpg', f'val/dogs/dog.{i}.jpg')
    os.rename(f'train/cat.{i}.jpg', f'val/cats/cat.{i}.jpg')

## **Model**

For this homework we will use Convolutional Neural Network (CNN. Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a covolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`


For clarification about kernel size and max pooling, check [Week #11 Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

### **Libs**

In [15]:
# !pip3 install numpy matplotlib

In [17]:
# !pip3 install --upgrade tensorflow

In [19]:
# check install
# !python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.optimizers import SGD, Adam
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten

In [22]:
def define_model():
    # Model
    inputs = keras.Input(shape=(150, 150, 3))
    conv = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
    pool = keras.layers.MaxPooling2D(pool_size=(2,2))(conv)
    vector = keras.layers.Flatten()(pool)
    dense = keras.layers.Dense(64, activation='relu')(vector)
    output = keras.layers.Dense(1, activation='sigmoid')(dense)
    model = keras.Model(inputs, output)

    # Optimizer=> single output problem => BinaryCrossentropy 
    optimizer = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
    loss = keras.losses.BinaryCrossentropy(from_logits=False)

    # Model compiling
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

## **Question 1**

Since we have a binary classification problem, what is the best loss function for us?

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`


> for binary classification problem, the better choice is **`BinaryCrossentropy`**

## **Question 2**

What's the total number of parameters of the model? You can use the `summary` method for that. 

In [23]:
model = define_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

**_answer_**
> **`11,215,873`**

## **Generators and Training**

For the next two questions, use the following data generator for both train and validation:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and validaition 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)
```

Note `validation_steps=50` - this parameter says "run only 50 steps on the validation data for evaluating the results". 
This way we iterate a bit faster, but don't use the entire validation dataset.
That's why it's important to shuffle the validation dataset as well. 

## **Question 3**

What is the median of training accuracy for this model?

In [24]:
train_gen = ImageDataGenerator(rescale = 1./255)
train_generator = train_gen.flow_from_directory(
    '/content/train/',
    target_size=(150, 150),
    classes= ['dogs', 'cats'],
    class_mode= 'binary',
    batch_size=20
)

Found 20000 images belonging to 2 classes.


In [26]:
val_gen = ImageDataGenerator(rescale = 1./255)
validation_generator = val_gen.flow_from_directory(
    '/content/val/',
    target_size=(150, 150),
    classes= ['dogs', 'cats'],
    class_mode= 'binary',
    batch_size=20
)

Found 5000 images belonging to 2 classes.


In [27]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 42s 414ms/step - loss: 0.6934 - accuracy: 0.5335 - val_loss: 0.6820 - val_accuracy: 0.5740
Epoch 2/10
100/100 [==============================] - 39s 390ms/step - loss: 0.6822 - accuracy: 0.5685 - val_loss: 0.6703 - val_accuracy: 0.5980
Epoch 3/10
100/100 [==============================] - 39s 388ms/step - loss: 0.6684 - accuracy: 0.6155 - val_loss: 0.6647 - val_accuracy: 0.5640
Epoch 4/10
100/100 [==============================] - 38s 385ms/step - loss: 0.6676 - accuracy: 0.5900 - val_loss: 0.6656 - val_accuracy: 0.5760
Epoch 5/10
100/100 [==============================] - 39s 386ms/step - loss: 0.6630 - accuracy: 0.5950 - val_loss: 0.6511 - val_accuracy: 0.6120
Epoch 6/10
100/100 [==============================] - 38s 384ms/step - loss: 0.6625 - accuracy: 0.6005 - val_loss: 0.6484 - val_accuracy: 0.6430
Epoch 7/10
100/100 [==============================] - 38s 381ms/step - loss: 0.6565 - accuracy: 0.6005 - val_loss: 0.6433 - val_ac

In [28]:
np.median(history.history['accuracy'])

0.6004999876022339

**_answer_**
> **`0.56`**

## **Question 4**

What is the standard deviation of training loss for this model?

In [31]:
np.std(history.history['loss'])

0.020323706097159108

**_answer_**
> **`0.01`**

## **Data Augmentation**

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [32]:
train_gen = ImageDataGenerator(
                               rescale=1./255,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest'
)

train_generator = train_gen.flow_from_directory(
    '/content/train/',
    target_size=(150, 150),
    classes= ['dogs', 'cats'],
    class_mode= 'binary',
    batch_size=20,
)

Found 20000 images belonging to 2 classes.


In [33]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 46s 464ms/step - loss: 0.6643 - accuracy: 0.5885 - val_loss: 0.6302 - val_accuracy: 0.6540
Epoch 2/10
100/100 [==============================] - 46s 461ms/step - loss: 0.6619 - accuracy: 0.5875 - val_loss: 0.6307 - val_accuracy: 0.6410
Epoch 3/10
100/100 [==============================] - 46s 463ms/step - loss: 0.6529 - accuracy: 0.6105 - val_loss: 0.6164 - val_accuracy: 0.6550
Epoch 4/10
100/100 [==============================] - 46s 464ms/step - loss: 0.6546 - accuracy: 0.6040 - val_loss: 0.6376 - val_accuracy: 0.6060
Epoch 5/10
100/100 [==============================] - 47s 465ms/step - loss: 0.6522 - accuracy: 0.6165 - val_loss: 0.6201 - val_accuracy: 0.6580
Epoch 6/10
100/100 [==============================] - 46s 463ms/step - loss: 0.6556 - accuracy: 0.6005 - val_loss: 0.6222 - val_accuracy: 0.6560
Epoch 7/10
100/100 [==============================] - 46s 463ms/step - loss: 0.6545 - accuracy: 0.6100 - val_loss: 0.6213 - val_ac

## **Question 5** 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of validation loss for the model trained with augmentations?

In [34]:
np.mean(history.history['val_loss'])

0.6250438749790191

**_answer_**
> **`0.67`**

## **Question 6**

What's the average of validation accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

In [35]:
np.mean(history.history['val_accuracy'][-5:])

0.6541999936103821

In [36]:
history.history['val_accuracy'][-5:]

[0.656000018119812,
 0.6449999809265137,
 0.6909999847412109,
 0.6380000114440918,
 0.640999972820282]

**_answer_**
> **`0.65`**

## **Submit the results**

Submit your results here: https://forms.gle/AE9v22MFnXxB9gyZ8

If your answer doesn't match options exactly, select the closest one.

## **Deadline**

The deadline for submitting is 22 November, 17:00 CET. After that, the form will be closed.
